In [1]:
import pymongo

In [2]:
# Conexión al servidor local de mongodb
conex = pymongo.MongoClient()
conex.list_database_names()

['admin', 'config', 'local', 'practica']

In [3]:
# Conexión a la base de datos
db = conex.practica

In [4]:
db.list_collection_names()

['documentos', 'autores']

## Consulta 1
Listado de todas las publicaciones de un autor determinado.

In [35]:
autor = "Daniel Gruss"
c1 = db.autores.find({"_id":autor}, {"publications":1})
c1 = list(c1)
print(c1[0]['publications'])

['tr/meltdown/s18', 'tr/meltdown/m18', 'journals/corr/abs-1807-10535', 'journals/corr/abs-1801-01207', 'journals/corr/GrussMM15', 'journals/corr/LippGSM15', 'journals/corr/abs-1902-03256', 'journals/corr/abs-1801-01203', 'journals/corr/abs-1905-05725', 'journals/corr/SchwarzLGWMSM17', 'journals/corr/Gruss17', 'journals/corr/SchwarzWGMM17', 'journals/corr/PesslGMM15', 'journals/corr/abs-1901-01161', 'journals/corr/abs-1905-05726', 'journals/corr/abs-1710-00551', 'journals/corr/GrussMW15', 'journals/corr/abs-1811-05441', 'journals/corr/abs-1905-09100', 'journals/corr/abs-1905-12701', 'journals/corr/abs-1805-04956', 'journals/corr/abs-1711-01254', 'journals/usenix-login/GrussHG18', 'journals/cybersec/SchwarzWGMM20', 'conf/ccs/Gruss0WGMML18', 'conf/ccs/0001LMBS0G19', 'conf/ccs/0001GLMSFM18', 'conf/ccs/VeenFLGMVBRG16', 'conf/ccs/GrussMFLM16', 'conf/ccs/GrussKT0THIF19', 'conf/ccs/CanellaGGGLMMP019', 'conf/ccs/SpreitzerKGM18', 'conf/fc/SchwarzMGM17', 'conf/uss/GrussLSOHC17', 'conf/uss/Lipp0G0

## Consulta 2
Número de publicaciones de un autor determinado.

In [76]:
autor = "Daniel Gruss"
c2 = db.autores.aggregate([
    {"$match":{"_id":autor}},
    {"$project":{"Número de publicaciones":{"$size":"$publications"}}
    }
])

print(list(c2))

[{'_id': 'Daniel Gruss', 'Número de publicaciones': 57}]


## Consulta 3
Número de artículos en revista para el año 2018.

In [64]:
c3 = db.documentos.aggregate([
    {"$sample":{"size":20000}},
    {"$match":{"year":"2018", "type":"article"}},
    {"$count":"Número artículos en el año 2018"}
])
print(list(c3))

[{'Número artículos en el año 2018': 836}]


## Consulta 4
Número de autores ocasionales, es decir, que tengan menos de 5 publicaciones en total.

In [94]:
c4 = db.autores.aggregate([
    {"$sample":{"size":20000}},
    {"$project":{"N_publicaciones":{"$size":"$publications"}}},
    {"$match":{"N_publicaciones":{"$lt":5}}},
    {"$count":"Número de autores ocasionales"}
])
print(list(c4))

[{'Número de autores ocasionales': 14864}]


## Consulta 5
Número de artículos de revista (article) y número de artículos en congresos
(inproceedings) de los diez autores con más publicaciones totales.

In [82]:
c5 = db.autores.aggregate([
    {"$sample":{"size":20000}},
    {"$project":{"N_publicaciones":{"$size":"$publications"}, "publications": 1}},
    {"$sort":{"N_publicaciones":-1}},
    {"$limit":10},
    {"$unwind": "$publications"},
    {"$lookup":
     {
       "from":"documentos",
       "localField":"publications",
       "foreignField":"_id",
       "as":"ref_publication"
     }},
  
])
c5 = list(c5)
print(c5)


SyntaxError: invalid syntax (<ipython-input-82-b4bbf799b296>, line 15)

## Consulta 6
Número medio de autores de todas las publicaciones que tenga en su conjunto de datos.

In [1]:
c6 = db.documentos.aggregate([
    {"$sample":{"size":10000}},
    {"$project":{"N_autores":{"$size":"$authors"}}},
    {"$group":{
        "_id":None, 
        "Númeno medio de autores":{"$avg":"$N_autores"}}},
    {"$project":{"_id":0}}
])

print(list(c6))

NameError: name 'db' is not defined

## Consulta 7
Listado de coautores de un autor (Se denomina coautor a cualquier persona que haya
firmado una publicación).

In [77]:
autor = "Daniel Gruss"
c7 = db.documentos.aggregate([
    {"$match":{"authors":autor}},
    {"$unwind": "$authors"},
    {"$group":{
        "_id":None, 
        "Listado":{"$addToSet":"$authors"}}},
    {"$project": {"Listado de coautores": 
                      {"$setDifference": ["$Listado", [autor]]}, "_id":0}}
])

print(list(c7))

[{'Listado de coautores': ['Luca Mayr', 'Florian Lackner', 'Manuel Weber', 'Thomas Unterluggauer', 'Mario Werner', 'Jann Horn', 'Istvan Haller', 'Felix Schuster', 'Julian Lettner', 'Felix Kirchengast', 'Martina Lindorfer', 'Victor van der Veen', 'Stefan More', 'Yanick Fratantonio', 'Timo Malderle', 'Jason Hennessey', 'Raphael Spreitzer', 'Klaus Wagner 0001', 'Richard Fellner', 'Frank Piessens', 'Erik Kraft', 'Cristiano Giuffrida', 'Alex Ionescu', 'Jo Van Bulck', 'Kay Romer', 'Peter Pessl', 'Lukas Giner', 'Benjamin von Berg', 'Claudio Canella', 'Herbert Bos', 'Misiker Tadesse Aga', "Sioli O'Connell", 'Lukas Raab', 'David Bidner', 'Trishita Tiwari', 'Clementine Maurice', 'Jonas Juffinger', 'Manuel Costa', 'Werner Haas', 'Stefan Mangard', 'Paul Kocher', 'Moritz Lipp', 'Ari Trachtenberg', 'Thomas Prescher 0002', 'Jon Masters', 'Julian Stecklina', 'Markus Eger', 'Berk Sunar', 'Daniel Genkin', 'Daniel Moghimi', 'Matthias Wubbeling', 'Giovanni Vigna', 'Michael Schwarz 0001', 'Lukas Lamster', 

## Consulta 8
Edad de los 5 autores con un periodo de publicaciones más largo (Se considera la Edad
de un autor al número de años transcurridos desde la fecha de su primera publicación
hasta la última registrada).

In [14]:
c8 = db.autores.aggregate([
    {"$sample":{"size":200}},
    {"$unwind": "$publications"},
    {"$lookup":
     {
       "from":"documentos",
       "localField":"publications",
       "foreignField":"_id",
       "as":"ref_publication"
     }},
    {"$addFields":{"año_min":{"$min":"$ref_publication.year"}}},
    {"$addFields":{"año_max":{"$max":"$ref_publication.year"}}},
    {"$addFields": {"Ultimo": {"$toInt": "$año_max"}, "Primero": {"$toInt": "$año_min"}}}
    # {"$addFields":{"edad":{"$subtract":["año_min","año_max"]}}},
    
    
])
print(list(c8))

OperationFailure: Unrecognized expression '$toInt'

## Consulta 9
Número de autores novatos, es decir, que tengan una Edad menor de 5 años.

## Consulta 10
Porcentaje de publicaciones en revistas con respecto al total de publicaciones.

In [10]:
c10 = db.documentos.aggregate([
        {"$project": {"_id": 1, "articulo": {"$cond": [{"$eq": ["$type","article"]},1,0]}}},
        {"$group":{
        "_id":None, 
        "Número de artículos":{"$sum":"$articulo"}, 
        "Número de documentos":{"$sum":1}}},
        {"$project":{"_id":0, "Porcentaje":{"$divide":[{"$multiply":["$Número de artículos",100]},"$Número de documentos"]}}}      
])

print(list(c10))

[{'Porcentaje': 45.89830331525446}]
